# Fastapi Client Simulation

Este notebook permite utilizar la API de predicción de cáncer de mama, tanto en **local** como en la nube mediante **Render**, para realizar esto se define una función **prediction_request** la cual realiza una petición POST a la API de predicción y maneja la respuesta.

---
⚠️NOTA:  
Antes de ejecutar las pruebas con la API en **local**, asegúrate de que el servidor esté corriendo. La guía rápida de configuración local se encuentra en el archivo **README** del proyecto.

---

In [1]:
# Librerías necesarias
import requests
import json

In [2]:
# Opciones de conexión
local_api_url = "http://127.0.0.1:8000/predict" # Requiere que el servidor esté corriendo localmente
render_api_url = "https://fastapi-breast-cancer-predictor-uyv6.onrender.com/predict"

## Casos de prueba

Se incluyen tres tipos de casos:

1) Caso válido – JSON con datos correctos.

2) Caso válido alternativo – Valores distintos para probar variedad.

3) Caso inválido – Datos que deberían producir error (ej. mean_area negativo).

**Puedes añadir más casos de prueba agregando y/o modificando los JSON de ejemplo.**

In [3]:
# JSON de entradas para pruebas
# Primer caso válido
data_valid = {
    "mean_radius": 14.127292,
    "mean_texture": 19.289649,
    "mean_perimeter": 91.969033,
    "mean_area": 654.889104,
    "mean_smoothness": 0.096360,
    "mean_compactness": 0.104341,
    "mean_concavity": 0.088799,
    "mean_concave_points": 0.048919,
    "mean_symmetry": 0.181162,
    "mean_fractal_dimension": 0.062798,
    "radius_error": 0.405172,
    "texture_error": 1.216853,
    "perimeter_error": 2.866059,
    "area_error": 40.337079,
    "smoothness_error": 0.007041,
    "compactness_error": 0.025478,
    "concavity_error": 0.031894,
    "concave_points_error": 0.011796,
    "symmetry_error": 0.020542,
    "fractal_dimension_error": 0.003795,
    "worst_radius": 16.269190,
    "worst_texture": 25.677223,
    "worst_perimeter": 107.261213,
    "worst_area": 880.583128,
    "worst_smoothness": 0.132369,
    "worst_compactness": 0.254265,
    "worst_concavity": 0.32,
    "worst_concave_points": 0.114606,
    "worst_symmetry": 0.2,
    "worst_fractal_dimension": 0.231
}


# Segundo caso válido (valores diferentes)
data_valid_alt = data_valid.copy()
data_valid_alt["mean_radius"] = 10.5
data_valid_alt["worst_area"] = 500

# Tercer caso inválido (mean_area negativo)
data_invalid = data_valid.copy()
data_invalid["mean_area"] = -50

## Uso de la API
Se declara una función test_request que envía un POST a la API.

In [ ]:
def prediction_request(url, name, payload):
    """"
    Realiza una petición POST a la API de predicción y maneja la respuesta.
    Args:
        url (str): URL de la API.
        name (str): Nombre del caso de prueba.
        payload (dict): Datos a enviar en la petición.
    """
    print(f"\n API Test: {name}")
    try:
        response = requests.post(url, json=payload)
        print("Status:", response.status_code)
        response.raise_for_status()  # Lanza excepción si status >= 400
        print("Response JSON:", json.dumps(response.json(), indent=2))
    except requests.exceptions.HTTPError:
        try:
            # Intentamos extraer el mensaje de error desde el JSON
            msg = response.json().get("detail") or response.json().get("message")
            print(f"Error {response.status_code}: {msg}")
        except Exception:
            print(f"Error {response.status_code}: No se pudo leer mensaje de la API")
    except requests.exceptions.RequestException as e:
        print("Error de conexión o request:", e)


## Ejecución de request

In [ ]:
# API LOCAL
print("=== API LOCAL ===")
prediction_request(local_api_url, "Local - Caso válido", data_valid)
prediction_request(local_api_url,"Local - Caso válido alternativo", data_valid_alt)
prediction_request(local_api_url, "Local - Caso inválido",data_invalid)

=== API LOCAL ===

 API Test: Local - Caso válido
Status: 200
Response JSON: {
  "prediction": 1,
  "probability": 0.7245078227783659
}

 API Test: Local - Caso válido alternativo
Status: 200
Response JSON: {
  "prediction": 1,
  "probability": 0.8652547561886609
}

 API Test: Local - Caso inválido
Status: 422
Error 422: ['Value error, Los valores no pueden ser negativos']


In [ ]:
# API Render
print("=== API RENDER ===")
prediction_request(render_api_url, "Render - Caso válido", data_valid)
prediction_request(render_api_url,"Render - Caso válido alternativo", data_valid_alt)
prediction_request(render_api_url, "Render - Caso inválido",data_invalid)


=== API RENDER ===

 API Test: Render - Caso válido
Status: 200
Response JSON: {
  "prediction": 1,
  "probability": 0.7245078227783659
}

 API Test: Render - Caso válido alternativo
Status: 200
Response JSON: {
  "prediction": 1,
  "probability": 0.8652547561886609
}

 API Test: Render - Caso inválido
Status: 422
Error 422: ['Value error, Los valores no pueden ser negativos']


## Conclusión

La API responde de forma consistente y robusta frente a:
- Datos correctos → entrega predicción.  
- Datos inválidos → entrega mensajes de error claros.  

El flujo de request–response fue validado tanto en **entorno local** como en **Render**.